<b style="font-size:2em;">Final Project Submission</b>

* Student name: Edward Amor 
* Student pace: part time
* Scheduled project review date/time: 
* Instructor name: Victor Geislinger
* Blog post URL: 

# Business Understanding

## Background

A real estate company in the King County area wants to investigate house sales in order to predict pricing. Our task is to clean, explore, and model the housing data they have supplied to us in `kc_house_data.csv`, and identify house features which significantly effect house price. 

## Scope

To identify which house features significantly effect house price, we will use standard EDA practices, hypothesis testing, and regression modeling to come to our conclusions.

For our client's consumption the results of our analysis will be condensed into a presentation. 

## Goal

Our goal is to generate a multivariate linear regression model which as accurately as possible predicts the sale price of houses.  

## Objectives

1. Data Acquisition
2. Data Understanding
  - Data Cleaning
  - Data Exploration
  - Data Visualization
3. Modeling
  - Feature Engineering
    - Feature Transformation
    - Feature Selection
  - Training
  - Model Evaluation

# Data Acquisition

The data for this project can be found in the file `kc_house_data.csv`.

Within this repository is also `data_dictionary.csv` which describes the data features.

# Data Understanding

According to our data dictionary the features of our dataset are as follows:

| column        | description                                                  |
| ------------- | ------------------------------------------------------------ |
| id            | unique identified for a house                                |
| date          | Date house was sold                                          |
| price         | Price is prediction target                                   |
| bedrooms      | Number of Bedrooms/House                                     |
| bathrooms     | Number of bathrooms/bedrooms                                 |
| sqft_living   | square footage of the home                                   |
| sqft_lot      | square footage of the lot                                    |
| floors        | Total floors (levels) in house                               |
| waterfront    | House which has a view to a waterfront                       |
| view          | Has been viewed                                              |
| condition     | How good the condition is ( Overall )                        |
| grade         | overall grade given to the housing unit                      |
| sqft_above    | square footage of house apart from basement                  |
| sqft_basement | square footage of the basement                               |
| yr_built      | Built Year                                                   |
| yr_renovated  | Year when house was renovated                                |
| zipcode       | zip                                                          |
| lat           | Latitude coordinate                                          |
| long          | Longitude coordinate                                         |
| sqft_living15 | The square footage of interior housing living space for the nearest 15 neighbors |
| sqft_lot15    | The square footage of the land lots of the nearest 15 neighbors |

Of the 20 columns most notable is the price feature which will be our target during linear regression.

Another thing to note is we have location data. This could be used to provide some insight into pricing by location, but may also be of use when delivering key information to our client.

## Data Cleaning

For this section we will inspect our dataset cleaning up inconsistencies such as, null values, duplicates, and incorrect data types.

In [ ]:
# import the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set()

In [ ]:
# read in dataframe and output dataframe info
df = pd.read_csv("kc_house_data.csv")
df.info()

Our raw dataset contains **21597 records**, and has **21 features**.

The data types found within our dataset are: `float64`, `int64`, `object`.

Next we will inspect our features more closely, especially the ones which are of `object` data type. Just to make sure they're the correct data type.

In [ ]:
# Output the first 5 rows
display(df.head(5))

The `date` column should be transformed to the `datetime` data type instead of `object`.

The `sqft_basement` column needs to be further inspected, the values appear to be `float` there may be some `string` values.

In [ ]:
# convert date column from string data type to a datetime
df['date'] = pd.to_datetime(df['date'])

In [ ]:
# inspect the values of the sqft_basement column
display(df['sqft_basement'].value_counts())

There is a '?' value, this may mean that the value is unknown. We will replace this with a null value, and then convert the column into a float.

In [ ]:
# replace ? with np.nan in the sqft_basement column and convert the column to float
df['sqft_basement'] = df['sqft_basement'].replace('?', np.nan).astype('float64')

In [ ]:
# redisplay the values in the sqft_basement column
display(df['sqft_basement'].value_counts())

All our columns are now numeric, it's time to inspect for any interesting anomalies.

### Inspecting Distributions

In [ ]:
# plot a distribution plot for each feature
fig, ax = plt.subplots(7, 3, figsize=(16, 9*3)) # 21 sub plots

for col, axes in zip(df.columns, ax.flatten()):
    sns.distplot(df[col], kde=False, rug=True, ax=axes)

plt.show()

Columns to disregard when cleaning:

- `id`
- `date`
- `long`
- `lat`
- `zipcode`

From the histograms it looks like we may want to drop the `yr_renovated` column, we will have to investigate further.

There are quite a few columns which should be transformed into the `categorical` data type.

### Impute Inconsistencies

#### yr_renovated

In [ ]:
# inspect the yr_renovated column by plotting a violinplot
fig, ax = plt.subplots(figsize=(16, 9))

sns.violinplot('yr_renovated', data=df, width=.5)
plt.show()

In [ ]:
# check the value counts for the y_renovated column
display(df['yr_renovated'].value_counts().head())

An extremely large percentage of the `yr_renovated` column has the value 0, meaning no renovation has occurred. Instead of removing the column, we can convert the values other than 0 to a 1. Giving us a usable feature instead of dropping one.

In [ ]:
# transform the values other than 0 to a 1 in yr_renovated, and rename the column
mask = df['yr_renovated'] != 0
df.loc[mask, 'yr_renovated'] = 1
df.rename(columns={'yr_renovated': 'renovated'}, inplace=True)

#### bedrooms

Another column we see an error in is the `bedrooms` columns. There is a huge skew, maybe due to a typing error.

In [ ]:
# output value counts
display(df['bedrooms'].value_counts())

The house with 33 bedrooms may be an typing error, most likely a room with 3 bedrooms, not 33 which would be highly unusual.

In [ ]:
# change the 33 bedroom house to a 3 bedroom
df.loc[df['bedrooms'] == 33, 'bedrooms'] = 3

### NA Values

In [ ]:
# Output how many NA values we have
display(df.isna().sum())

For the `waterfront` and `view` columns, we will replace the na values with 0, to signify no waterfront and no views respectively.

For the `sqft_basement` column we will replace the na values with 0, to signify there is no basement.

In [ ]:
# replace the na values with 0 in waterfront, view, sqft_basement
df.replace(np.nan, 0, inplace=True)
display(df.isna().sum())

### Outliers

Since we've taken care of our null values, now we can focus on taking care of outliers which will significantly affect our linear regression in the future.

We will first start with our `price` feature.

In [ ]:
# show boxplot of price column
fig, ax = plt.subplots(figsize=(16, 5))

sns.boxplot('price', data=df, ax=ax, showmeans=True)

plt.show()

There appears to be a significant amount of outliers. Let's identify how many, and whether we should eliminate them.

In [ ]:
# using iqr find outliers
q1, q3 = np.quantile(df['price'], (.25, .75))
iqr = q3 - q1
low_bd, up_bd = q1 - iqr*1.5, q3 + iqr*1.5 # upper and lower bound of range
mask = (df['price'] < low_bd) | (df['price'] > up_bd) # outliers mask

outliers_mask = df.index[mask].to_numpy()

In [ ]:
# count our outliers
display(f"Outliers Count: {mask.sum()}")
display(f"Outliers Percentage: {mask.sum()/df.shape[0]:.0%}")

Since we would only be removing approximately 5% of our dataset, 1158 data points, we will drop those rows.

In [ ]:
# drop the data points which are price outliers and display a new boxplot of price
df.drop(index=outliers_mask, inplace=True, errors='ignore') # ignore to allow re run

fig, ax = plt.subplots(figsize=(16, 5))

sns.boxplot(df['price'])
plt.show()

We've successfully removed the large amount of `price` outliers that were previously in our dataset.

### Type Conversion

We will convert columns which should be categorical to their correct type.

- bedrooms
- bathrooms
- floors
- waterfront
- view
- condition
- grade
- renovated
- zipcode

In [ ]:
# access and convert the categorical columns
cat_cols = ['bedrooms', 'bathrooms', 'floors', 'waterfront', 'view',
           'condition', 'grade', 'renovated', 'zipcode']

df.loc[:, cat_cols] = df.loc[:, cat_cols].astype('category')

In [ ]:
# verify conversion
display(df.dtypes)

## Data Exploration

We'll use some guided questions to help us explore our data easier.

1. Using location data, where are the highest valued properties?
2. Are waterfront properties significantly expensive than non waterfront properties?

### Location

In [ ]:
# import our libraries
import plotly.express as px
from plotly.offline import init_notebook_mode

# connect to online cdn
init_notebook_mode(connected=True)

In [ ]:
# plot pricing of houses across King County
fig = px.scatter_mapbox(df.sort_values('price'), lat='lat', lon='long', color='price',
                       color_continuous_scale=px.colors.sequential.BuGn,
                       mapbox_style='carto-positron',
                       center=dict(lat=df.lat.median(), lon=df.long.median()),
                       title="House Prices in King County")
fig.show()

There appears to be higher concentrations of high valued houses towards the north of King County, With the majority of those located on the waterfront, there may be other contributing factors since there are high valued houses scattered around King County.

Let's inspect by zip code the median house price, to get a better picture at which areas have higher valued houses.

In [ ]:
# import json
import json

In [ ]:
# load in geojson of king county zip codes
with open('kc_zips.geojson') as f:
    geojson = json.load(f)

The King County zip codes geojson data was compiled from a larger dataset containing all US zip codes. The larger dataset can be found at: https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2019.html

The downloaded data was then converted to a 1.4 GB geojson file by using https://mapshaper.org/. After which it was filtered for only zip codes in King County. 

In [ ]:
# create grouped dataframe with median house price for each zip code
zipcode_df = df.groupby('zipcode').median().reset_index()
zipcode_df['zipcode'] = zipcode_df['zipcode'].astype(str)

In [ ]:
# plot median house price data on map by zipcode
fig = px.choropleth_mapbox(zipcode_df, geojson=geojson, 
                          featureidkey='properties.ZCTA5CE10', locations='zipcode', 
                          color='price', mapbox_style='carto-positron', opacity=.3,
                          center=dict(
                              lat=zipcode_df.lat.median(),
                              lon=zipcode_df.long.median()
                          ), color_continuous_scale=px.colors.sequential.BuGn,  
                          title="King County Median House Price by Zip Code")
fig.show()

The map shows us that the 3 zip codes with the highest median house prices are located in the centrally. **Those 3 zip codes are 98039, 98040, and 98004**.

In the future it would be nice to reverse geocode the data we have, and see pricing by neighborhood.

### Waterfront Properties

In [ ]:
# immport our libraries
from scipy import stats

In [ ]:
# plot two distplots of the different waterfront price values
mask = df['waterfront'] == 1

fig, ax = plt.subplots(figsize=(16, 9))

sns.distplot(df.loc[mask, 'price'], norm_hist=True, ax=ax, label='waterfront')
sns.distplot(df.loc[~mask, 'price'], norm_hist=True, ax=ax, label='non-waterfront')

plt.show()

These two groups don't look too different, but we can use some hypothesis testing to verify our assumptions.

$\mu_0:$ Waterfront House Mean Value

$\mu_1:$ Non-Waterfront House Mean Value

$H_0: \mu_0 = \mu_1$

$H_a: \mu_0 \ge \mu_1$

$\alpha = $ 5%

In [ ]:
# seperate our two populations and also show some 5-point summary of them
mask = df['waterfront'] == 1
waterfront = df.loc[mask, 'price']
nonwaterfront = df.loc[~mask, 'price']

display(df.groupby('waterfront').describe()['price'])

Our means are different by about $200,000, but our standard deviations are approximately the same.

In [ ]:
# Run a welch's 2 sample t-test
results = stats.ttest_ind(waterfront, nonwaterfront, equal_var=False)
print(f"T-Stat: {results.statistic:.3f}", f"P-Value: {results.pvalue:.8%}", sep='\n')

It appears that we do have a statistically significantly difference between the two populations. For our stakeholders we will provide a simple visualization to drive the point home that waterfront properties are valued higher. 

In [ ]:
# plot barplot of the two populations average value wiht a confidence level of 95%
fig, ax = plt.subplots(figsize=(16, 9))

sns.barplot(x='waterfront', y='price', data=df, ax=ax)

plt.title('King County Average House Value of Waterfront/Non-Waterfront Properties')

plt.xticks([0, 1], ['Non-Waterfront', 'Waterfront'])
locs, labels = plt.yticks()
plt.yticks(locs, [f"{loc/1000:n}" for loc in locs])

plt.ylabel("Price (thousands USD)")
plt.xlabel("")
plt.show()